In [ ]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2 MB 22 kB/s 
     |████████████████████████████████| 198 kB 50.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=22fa10f47ca6b3ea41582ca07da72d9dd9f91ed64fb9c16c567944689384bd63
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [ ]:
!cd /usr/local/lib/python3.7/dist-packages/pyspark/jars && wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/1.2.20.1043/RedshiftJDBC42-no-awssdk-1.2.20.1043.jar

--2022-03-05 14:54:31--  https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/1.2.20.1043/RedshiftJDBC42-no-awssdk-1.2.20.1043.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 3.5.0.130
Connecting to s3.amazonaws.com (s3.amazonaws.com)|3.5.0.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2413910 (2.3M) [application/java-archive]
Saving to: ‘RedshiftJDBC42-no-awssdk-1.2.20.1043.jar’

RedshiftJDBC42-no-a 100%[===================>]   2.30M  3.57MB/s    in 0.6s    

2022-03-05 14:54:32 (3.57 MB/s) - ‘RedshiftJDBC42-no-awssdk-1.2.20.1043.jar’ saved [2413910/2413910]



In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
.appName('Python Spark SQL basic example')\
.config('spark.jars', '/usr/local/lib/python3.7/dist-packages/pyspark/jars/RedshiftJDBC42-no-awssdk-1.2.20.1043.jar')\
.getOrCreate()

In [ ]:
spark

In [ ]:
import pandas as pd

namegender_pd = pd.read_csv('https://s3-geospatial.s3-us-west-2.amazonaws.com/name_gender.csv')

In [ ]:
namegender_pd.head(5)

,name,gender
0,Adaleigh,F
1,Amryn,Unisex
2,Apurva,Unisex
3,Aryion,M
4,Alixia,F


In [ ]:
namegender_pd.groupby('gender').count()

,name
gender,
F,65
M,28
Unisex,7


In [ ]:
namegender_df = spark.createDataFrame(namegender_pd)

namegender_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)



In [ ]:
namegender_df.show()

+----------+------+
|      name|gender|
+----------+------+
|  Adaleigh|     F|
|     Amryn|Unisex|
|    Apurva|Unisex|
|    Aryion|     M|
|    Alixia|     F|
|Alyssarose|     F|
|    Arvell|     M|
|     Aibel|     M|
|   Atiyyah|     F|
|     Adlie|     F|
|    Anyely|     F|
|    Aamoni|     F|
|     Ahman|     M|
|    Arlane|     F|
|   Armoney|     F|
|   Atzhiry|     F|
| Antonette|     F|
|   Akeelah|     F|
| Abdikadir|     M|
|    Arinze|     M|
+----------+------+
only showing top 20 rows



In [ ]:
namegender_df.groupby(['gender']).count()

DataFrame[gender: string, count: bigint]

In [ ]:
namegender_df.groupby(['gender']).count().collect()

[Row(gender='F', count=65),
 Row(gender='M', count=28),
 Row(gender='Unisex', count=7)]

In [ ]:
#https://towardsdatascience.com/pyspark-and-sparksql-basics-6cb4bf967e53

In [ ]:
namegender_df.createOrReplaceTempView('namegender')

In [ ]:
namegender_group_df = spark.sql('SELECT gender, count(*) FROM namegender GROUP BY gender')

In [ ]:
namegender_group_df.collect()

[Row(gender='F', count(1)=65),
 Row(gender='M', count(1)=28),
 Row(gender='Unisex', count(1)=7)]

In [ ]:
namegender_group_df.show()

+------+--------+
|gender|count(1)|
+------+--------+
|     F|      65|
|     M|      28|
|Unisex|       7|
+------+--------+



In [ ]:
df_user_session_channel = spark.read \
                              .format('jdbc') \
                              .option('driver', 'com.amazon.redshift.jdbc42.Driver') \
                              .option('url', 'jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234') \
                              .option('dbtable', 'raw_data.user_session_channel') \
                              .load()

In [ ]:
df_session_timestamp = spark.read \
                              .format('jdbc') \
                              .option('driver', 'com.amazon.redshift.jdbc42.Driver') \
                              .option('url', 'jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234') \
                              .option('dbtable', 'raw_data.session_timestamp') \
                              .load()

In [ ]:
df_user_session_channel.createOrReplaceTempView('user_session_channel')
df_session_timestamp.createOrReplaceTempView('session_timestamp')

In [ ]:
channel_count_df = spark.sql("""
    SELECT channel, count(distinct userid) uniqueUsers
    FROM session_timestamp AS st
    LEFT JOIN user_session_channel AS usc ON st.sessionID = usc.sessionID
    GROUP BY 1
    ORDER BY 1
    """)

In [ ]:
channel_count_df

DataFrame[channel: string, uniqueUsers: bigint]

In [ ]:
channel_count_df.show()

+---------+-----------+
|  channel|uniqueUsers|
+---------+-----------+
| Facebook|        889|
|   Google|        893|
|Instagram|        895|
|    Naver|        882|
|  Organic|        895|
|  Youtube|        889|
+---------+-----------+

